In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor, StackingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score, mean_squared_error

# Load data
df = pd.read_csv("measures_v2.csv")

# Drop weak features
df = df.drop(columns=["ambient", "u_d"], errors='ignore')

# Define target and features
y = df['coolant']
X = df.drop(columns=['coolant', 'stator_yoke', 'stator_tooth', 'stator_winding', 'pm'], errors='ignore')

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Base models
base_models = [
    ('lr', make_pipeline(StandardScaler(), LinearRegression())),
    ('rf', RandomForestRegressor(n_estimators=100, random_state=42)),
    ('mlp', make_pipeline(StandardScaler(), MLPRegressor(hidden_layer_sizes=(64, 64), max_iter=500, random_state=42)))
]

# Final estimator
meta_model = Ridge()

# Ensemble model
ensemble = StackingRegressor(estimators=base_models, final_estimator=meta_model)
ensemble.fit(X_train_scaled, y_train)

# Predict and evaluate
y_pred = ensemble.predict(X_test_scaled)
r2 = r2_score(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)

print("📈 Ensemble Model Performance")
print("✅ R² Score:", r2)
print("✅ RMSE:", rmse)
